In [12]:
library(dplyr)

In [13]:
my_db <- src_sqlite("my_db.sqlite3", create = T)

src --> Collection of types of database tables. Examples include src_sqlite(), src_mysql(), src_postgres(), src_bigquery(). These are the types of database tables supported by dplyr

In [14]:
install.packages("nycflights13", repos="http://cran.us.r-project.org/")

Installing package into 'C:/Users/Anirudh/Documents/R/win-library/3.3'
(as 'lib' is unspecified)
Warning message:
"package 'nycflights13' is in use and will not be installed"

In [15]:
library(nycflights13)

In [ ]:
flights_sqlite <- copy_to(my_db, flights, temporary = FALSE, indexes = list(c("year", "month", "day"), "carrier", "tailnum"))

In [ ]:
flights_sqlite <- tbl(nycflights13_sqlite(), "flights")

In [ ]:
flights_sqlite

A way of getting "flights" table data

In [ ]:
tbl(my_db, sql("SELECT * FROM flights"))

Another way of getting "flights" table data throough SQL Query

In [ ]:
select(flights_sqlite, year:day, dep_delay, arr_delay)

To get only few columns

In [ ]:
filter(flights_sqlite, dep_delay > 240)

To get the data on a specified condition

In [ ]:
arrange(flights_sqlite, year, month, day)

In [ ]:
mutate(flights_sqlite, speed = air_time / distance)

In [ ]:
summarise(flights_sqlite, delay = mean(dep_time))

In [ ]:
c1 <- filter(flights_sqlite, year == 2013, month == 1, day == 1)

In [ ]:
c2 <- select(c1, year, month, day, carrier, dep_delay, air_time, distance)

In [ ]:
c3 <- mutate(c2, speed = distance / air_time * 60)

In [ ]:
c4 <- arrange(c3, year, day, carrier)

In [ ]:
c4

In [ ]:
collect(c4)

collect return all rows of a table in data fram format

In [ ]:
c4$query

In [ ]:
explain(c4)

In [ ]:
translate_sql(x)

In [ ]:
translate_sql("x")

In [ ]:
translate_sql(x == 1 && (y < 2 || z > 3))

In [ ]:
translate_sql(x ^ 2 < 10)

In [ ]:
translate_sql(x %% 2 == 10)

In [ ]:
translate_sql(1)

In [ ]:
translate_sql(1L)

In [ ]:
translate_sql(mean(x, trim = T))

In [ ]:
translate_sql(glob(x, y))

In [ ]:
translate_sql( x%like% "ab*")

In [ ]:
by_tailnum <- group_by(flights_sqlite, tailnum)
delay <- summarise(by_tailnum,
                  count = n(), dist = mean(distance), delay = mean(arr_delay))
delay <- filter(delay, count > 20, dist < 2000)
delay_local <- collect(delay)

flights_postgres <- tbl(src_postgres("nycflights13"), "flights")

daily <- group_by(flights_postgres, year, month, day)

bestworst <- daily %>% select(flight, arr_delay) %>% filter(arr_delay == min(arr_delay) || arr_delay == max(arr_delay))
bestworst$query

ranked <- daily %>%
        select(arr_delay) %>%
        mutate(rank = rank(desc(arr_delay)))
ranked$query